# Food Intake 

Ativando uma conexão de banco de dados em memória usando o SGBD H2:

In [1]:
%defaultDatasource jdbc:h2:mem:db

## Conjunto de tabelas para consultas básicas

In [2]:
DROP TABLE IF EXISTS Intake;

CREATE TABLE Intake (
  SeqN INTEGER NOT NULL,
  DayCode TINYINT NOT NULL,
  DraBF TINYINT,
  FCID_Code VARCHAR(10),
  Cooked_Status TINYINT,
  Food_Form TINYINT,
  Cooking_Method TINYINT,
  Intake DECIMAL(13,7),
  Intake_BW DECIMAL(13,10),
  PRIMARY KEY(SeqN, DayCode, FCID_Code, Cooked_Status, Food_Form, Cooking_Method)
) AS SELECT
  SEQN, DAYCODE, DRABF, FCID_Code, Cooked_Status, Food_Form, Cooking_Method, Intake,Intake_BW
FROM CSVREAD('../../data/food-intake/consumption/Commodity_CSFFM_Intake_0510-cropped.csv');

In [3]:
DROP TABLE IF EXISTS Recipes;

CREATE TABLE Recipes (
  Food_Code VARCHAR(8),
  Mod_Code VARCHAR(8),
  Ingredient_Num TINYINT,
  FCID_Code VARCHAR(10),
  Cooked_Status TINYINT,
  Food_Form TINYINT,
  Cooking_Method TINYINT,
  Commodity_Weight DECIMAL(5, 2),
  CSFII_9498_IND TINYINT,
  WWEIA_9904_IND TINYINT,
  WWEIA_0510_IND TINYINT,
  PRIMARY KEY(Food_Code, Mod_Code, Ingredient_Num)
) AS SELECT
  Food_Code, Mod_Code, Ingredient_Num, FCID_Code, Cooked_Status, Food_Form, Cooking_Method,
  Commodity_Weight, CSFII_9498_IND, WWEIA_9904_IND, WWEIA_0510_IND
FROM CSVREAD('../../data/food-intake/recipes/Recipes_WWEIA_FCID_0510.csv');

In [4]:
DROP TABLE IF EXISTS FCID_Description;

CREATE TABLE FCID_Description (
  CGN VARCHAR(2),
  CG_Subgroup VARCHAR(6),
  FCID_Code VARCHAR(10),
  FCID_Desc VARCHAR(55),
  PRIMARY KEY (FCID_Code)
) AS SELECT
  cgn, CG_Subgroup, FCID_Code, FCID_Desc
FROM CSVREAD('../../data/food-intake/basics/FCID_Code_Description.csv');

# SELECT Básico

### Todos os Táxis com todos os atributos

In [5]:
SELECT * FROM Intake LIMIT 10;

In [6]:
SELECT * FROM Recipes LIMIT 10;

In [7]:
SELECT * FROM Recipes WHERE Food_Code = '27111300' AND Mod_Code = 0;

In [8]:
SELECT SUM(Commodity_Weight) FROM Recipes WHERE Food_Code = '27111300' AND Mod_Code = 0;

99.79

In [9]:
SELECT MAX(SeqN) FROM Intake;

32776

In [10]:
SELECT COUNT(DISTINCT Food_Code) FROM Recipes;

7154

In [11]:
DROP VIEW IF EXISTS Participation_Profile;

CREATE VIEW Participation_Profile AS
SELECT R.FCID_Code, COUNT(DISTINCT (R.Food_Code, R.Mod_Code)) Participation
FROM Recipes R
GROUP BY R.FCID_Code;

SELECT * FROM Participation_Profile LIMIT 10;

In [12]:
SELECT COUNT(*) FROM Intake;

100000

In [13]:
SELECT COUNT(DISTINCT SEQN) FROM Intake;

1489

In [14]:
DROP VIEW IF EXISTS Commodity_Profile;

CREATE VIEW Commodity_Profile
AS SELECT I.FCID_Code, F.FCID_Desc, F.CGN, F.CG_Subgroup, COUNT(DISTINCT I.SeqN) Popularity,
          SUM(I.Intake) Intake_Sum, AVG(I.Intake) Intake_AVG,
          AVG(I.Intake_BW) Intake_BW_AVG, P.Participation Recipes
FROM Intake I, FCID_Description F, Participation_Profile P
WHERE I.FCID_Code = F.FCID_Code AND I.FCID_Code = P.FCID_Code
GROUP BY I.FCID_Code;

In [15]:
SELECT SUM(Consumption) FROM Commodity_Profile;

org.h2.jdbc.JdbcSQLException:  Column "CONSUMPTION" not found; SQL statement

In [16]:
CALL CSVWRITE('../../data/food-intake/computed/commodity-profile.csv',
              'SELECT * FROM Commodity_Profile');

368

### Taxis fabricados depois do ano 2000

In [17]:
SELECT * FROM Taxi WHERE AnoFab > 2000;

org.h2.jdbc.JdbcSQLException:  Table "TAXI" not found; SQL statement

### Placas que comecem com DK

In [18]:
SELECT * FROM Taxi WHERE placa LIKE 'DK%';

org.h2.jdbc.JdbcSQLException:  Table "TAXI" not found; SQL statement

### Placas com '7' na penultima posicao

In [19]:
SELECT * FROM Taxi WHERE placa LIKE '%7_';

org.h2.jdbc.JdbcSQLException:  Table "TAXI" not found; SQL statement

# Produto Cartesiano

### Produto Cartesiano entre Clientes e Corridas

In [20]:
SELECT Cliente.CliId, Cliente.Nome, Corrida.Placa, Corrida.DataPedido
FROM Cliente, Corrida

org.h2.jdbc.JdbcSQLException:  Table "CLIENTE" not found; SQL statement

# JOIN Implícito

### Cientes e as respectivas corridas - Join Implícito

In [21]:
SELECT Cliente.CliId, Cliente.Nome, Corrida.Placa, Corrida.DataPedido
FROM Cliente, Corrida
WHERE Cliente.CliId = Corrida.CliId

org.h2.jdbc.JdbcSQLException:  Table "CLIENTE" not found; SQL statement

### Modelo de taxi para cada corrida

In [22]:
SELECT Corrida.DataPedido, Corrida.Placa, Taxi.Modelo
       FROM Corrida, Taxi
       WHERE Corrida.Placa = Taxi.Placa;

org.h2.jdbc.JdbcSQLException:  Table "CORRIDA" not found; SQL statement

### Modelos de Táxi por Cliente (estágio 1)

In [23]:
SELECT Cliente.Nome, Corrida.DataPedido, Corrida.Placa, Taxi.Modelo
       FROM Cliente, Corrida, Taxi
       WHERE Cliente.CliId = Corrida.CliId AND Corrida.Placa = Taxi.Placa;

org.h2.jdbc.JdbcSQLException:  Table "CLIENTE" not found; SQL statement

### Modelos de Táxi por Cliente (estágio 2)

In [24]:
SELECT DISTINCT Cliente.Nome, Taxi.Modelo
       FROM Cliente, Corrida, Taxi
       WHERE Cliente.CliId = Corrida.CliId AND Corrida.Placa = Taxi.Placa;

org.h2.jdbc.JdbcSQLException:  Table "CLIENTE" not found; SQL statement

## ORDER BY

### Nome dos clientes ordenado alfabeticamente

In [25]:
SELECT Nome
       FROM Cliente
       ORDER BY Nome;

org.h2.jdbc.JdbcSQLException:  Table "CLIENTE" not found; SQL statement

### Modelos de taxi e os clientes que os tomaram

Ordena por Modelo, mas não por Cliente:

In [26]:
SELECT DISTINCT Cl.Nome, T.Modelo
       FROM Cliente Cl, Corrida Co, Taxi T
       WHERE Cl.CliId = Co.CliId AND
             Co.Placa = T.Placa
       ORDER BY T.Modelo;

org.h2.jdbc.JdbcSQLException:  Table "CLIENTE" not found; SQL statement

Ordena por Modelo e, para cada Modelo, por Cliente:

In [27]:
SELECT DISTINCT Cl.Nome, T.Modelo
       FROM Cliente Cl, Corrida Co, Taxi T
       WHERE Cl.CliId = Co.CliId AND
             Co.Placa = T.Placa
       ORDER BY T.Modelo, Cl.Nome;

org.h2.jdbc.JdbcSQLException:  Table "CLIENTE" not found; SQL statement

### Cliente e os modelos de taxi tomados
Ordena por Cliente, mas não por Modelo:

In [28]:
SELECT DISTINCT Cl.Nome, T.Modelo
       FROM Cliente Cl, Corrida Co, Taxi T
       WHERE Cl.CliId = Co.CliId AND
             Co.Placa = T.Placa
       ORDER BY Cl.Nome;

org.h2.jdbc.JdbcSQLException:  Table "CLIENTE" not found; SQL statement

Ordena por Cliente e, para cada Cliente, por Modelo

In [29]:
SELECT DISTINCT Cl.Nome, T.Modelo
       FROM Cliente Cl, Corrida Co, Taxi T
       WHERE Cl.CliId = Co.CliId AND
             Co.Placa = T.Placa
       ORDER BY Cl.Nome, T.Modelo;

org.h2.jdbc.JdbcSQLException:  Table "CLIENTE" not found; SQL statement

## JOIN explícito

### Clientes e respectivas corridas (para clientes que fizeram corrida)

In [30]:
SELECT Cl.CliId, Cl.Nome,
       Co.Placa, Co.DataPedido
       FROM Cliente Cl JOIN Corrida Co
            ON Cl.CliId = Co.CliId;

org.h2.jdbc.JdbcSQLException:  Table "CLIENTE" not found; SQL statement

### Taxis e respectivas corridas (para taxis que fizeram corrida)

In [31]:
SELECT Tx.placa, Co.cliid
       FROM Taxi Tx JOIN Corrida Co
            ON Tx.placa = Co.placa;

org.h2.jdbc.JdbcSQLException:  Table "TAXI" not found; SQL statement

## NATURAL JOIN

In [32]:
SELECT Tx.placa, Co.cliid
       FROM Taxi Tx
            NATURAL JOIN Corrida Co;

org.h2.jdbc.JdbcSQLException:  Table "TAXI" not found; SQL statement

## LEFT JOIN

### Clientes e respectivas corridas (para todos os clientes)

In [33]:
SELECT Cl.CliId, Cl.Nome,
       Co.Placa, Co.DataPedido
       FROM Cliente Cl LEFT JOIN Corrida Co
            ON Cl.CliId = Co.CliId;

org.h2.jdbc.JdbcSQLException:  Table "CLIENTE" not found; SQL statement

### Taxis e respectivas corridas (para todos os taxis)

In [34]:
SELECT Tx.placa, Co.cliid
       FROM Taxi Tx LEFT JOIN Corrida Co
            ON Tx.placa = Co.placa;

org.h2.jdbc.JdbcSQLException:  Table "TAXI" not found; SQL statement

## RIGHT JOIN

### Corridas e respectivos clientes (para todos os clientes)

In [35]:
SELECT Co.placa, Cl.nome
       FROM Corrida Co RIGHT JOIN Cliente Cl
            ON Co.cliid = Cl.cliid;

org.h2.jdbc.JdbcSQLException:  Table "CORRIDA" not found; SQL statement

# GROUP BY

### Modelos de Táxi cadastrados

In [36]:
SELECT T.Modelo
       FROM Taxi T
       GROUP BY T.Modelo;

org.h2.jdbc.JdbcSQLException:  Table "TAXI" not found; SQL statement

### Total de Táxis por Modelo

Modelos de Táxi cadastrados e quantos táxis há em cada modelo:

In [37]:
SELECT T.Modelo, COUNT(*)
       FROM Taxi T
       GROUP BY T.Modelo;

org.h2.jdbc.JdbcSQLException:  Table "TAXI" not found; SQL statement

## Agregando em mais de um nível

### Modelos de taxi tomados por cada cliente (agrupando por Cliente e por Modelo)

In [38]:
SELECT Cl.Nome, T.Modelo
       FROM Cliente Cl, Corrida Co, Taxi T
       WHERE Cl.CliId = Co.CliId AND
             Co.Placa = T.Placa;

org.h2.jdbc.JdbcSQLException:  Table "CLIENTE" not found; SQL statement

In [39]:
SELECT DISTINCT Cl.Nome, T.Modelo
       FROM Cliente Cl, Corrida Co, Taxi T
       WHERE Cl.CliId = Co.CliId AND
             Co.Placa = T.Placa;

org.h2.jdbc.JdbcSQLException:  Table "CLIENTE" not found; SQL statement

In [40]:
SELECT Cl.Nome, T.Modelo
       FROM Cliente Cl, Corrida Co, Taxi T
       WHERE Cl.CliId = Co.CliId AND
             Co.Placa = T.Placa
       GROUP BY Cl.Nome, T.Modelo;

org.h2.jdbc.JdbcSQLException:  Table "CLIENTE" not found; SQL statement

In [41]:
SELECT Cl.Nome, T.Modelo, COUNT(*)
       FROM Cliente Cl, Corrida Co, Taxi T 
       WHERE Cl.CliId = Co.CliId AND 
             Co.Placa = T.Placa
       GROUP BY Cl.Nome, T.Modelo;

org.h2.jdbc.JdbcSQLException:  Table "CLIENTE" not found; SQL statement

## Funções de agregação sem agrupamento

### Média de ano de fabricação para todos os Táxis

Quando é usada função de agregação sem especificação de agrupamento, toda a tabela é agregada como se fosse um único grupo.

In [42]:
SELECT Modelo, AnoFab FROM Taxi;

org.h2.jdbc.JdbcSQLException:  Table "TAXI" not found; SQL statement

In [43]:
SELECT Modelo, MIN(AnoFab), MAX(AnoFab), AVG(AnoFab)
       FROM Taxi
       GROUP BY Modelo;

org.h2.jdbc.JdbcSQLException:  Table "TAXI" not found; SQL statement

In [44]:
-- Maior ano de fabricação de toda a tabela
SELECT MAX(AnoFab) FROM Taxi;

org.h2.jdbc.JdbcSQLException:  Table "TAXI" not found; SQL statement

## Campos mostrados devem corresponder a agregação

Exemplo sem agregação:

In [45]:
SELECT Modelo, AnoFab
       FROM Taxi;

org.h2.jdbc.JdbcSQLException:  Table "TAXI" not found; SQL statement

### Modelo e Ano de Fabricação

Este exemplo com agregação produzirá um erro pois `AnoFab` (ano de fabricação) não foi agregado, portanto, pode haver mais de um ano de fabricação para o mesmo modelo.

In [46]:
SELECT Modelo, AnoFab
       FROM Taxi
       GROUP BY Modelo;

org.h2.jdbc.JdbcSQLException:  Table "TAXI" not found; SQL statement

### Modelo e Maior Ano de Fabricação
#### Corrigindo com função de agregação

Este exemplo escolhe o maior ano para cada Modelo agregado.

In [47]:
SELECT Modelo, MAX(AnoFab)
       FROM Taxi
       GROUP BY Modelo;

org.h2.jdbc.JdbcSQLException:  Table "TAXI" not found; SQL statement

# HAVING

### Número de Táxis por Modelo somente para os Táxis com ano de fabricação abaixo de 2000

Exemplo com `WHERE` em que a condição é aplicada antes de se agregar:

In [48]:
-- Todos os Táxis
SELECT Modelo, AnoFab
       FROM Taxi;

org.h2.jdbc.JdbcSQLException:  Table "TAXI" not found; SQL statement

In [49]:
SELECT Modelo, AnoFab
       FROM Taxi
       WHERE AnoFab < 2000;

org.h2.jdbc.JdbcSQLException:  Table "TAXI" not found; SQL statement

In [50]:
-- Selecionados e Agregados
SELECT Modelo, COUNT(*) NUM
       FROM Taxi
       WHERE AnoFab < 2000
       GROUP BY Modelo;

org.h2.jdbc.JdbcSQLException:  Table "TAXI" not found; SQL statement

### Modelos de Táxi que têm mais de um Táxi do respectivo modelo

Exemplo com `HAVING` em que a condição é aplicada após se agregar:

In [51]:
-- Agrupamento
SELECT Modelo, COUNT(*) NUM
       FROM Taxi
       GROUP BY Modelo;

org.h2.jdbc.JdbcSQLException:  Table "TAXI" not found; SQL statement

In [52]:
-- Agrupamento com HAVING
SELECT Modelo, COUNT(*) NUM
       FROM Taxi
       GROUP BY Modelo
       HAVING NUM > 1;

org.h2.jdbc.JdbcSQLException:  Table "TAXI" not found; SQL statement

## `HAVING` apenas para campos agregados 

### Modelos daqueles Táxis com ano de fabricação acima de 2000

A seleção a seguir produzirá um erro pois está se aplicando uma condição `HAVING` para um campo não agregado. Deveria ter sido usado `WHERE` como foi feito anteriormente.

In [53]:
SELECT Modelo, COUNT(*) NUM
       FROM Taxi
       GROUP BY Modelo
       HAVING AnoFab > 2000;

org.h2.jdbc.JdbcSQLException:  Table "TAXI" not found; SQL statement

## Exemplo de `HAVING` com funções de agregação

### Modelos de Táxi cujo menor Ano de Fabricação seja após 2000

In [54]:
-- sem agregacao (nao eh possível resolver a questao)
SELECT Modelo, AnoFab
       FROM Taxi
       WHERE AnoFab < 2000;

org.h2.jdbc.JdbcSQLException:  Table "TAXI" not found; SQL statement

In [55]:
-- testando após a agregação
SELECT Modelo, MAX(AnoFab) MaiorAno
       FROM Taxi
       GROUP BY Modelo
       HAVING MaiorAno < 2000;

org.h2.jdbc.JdbcSQLException:  Table "TAXI" not found; SQL statement

# VIEW

### Total de Táxis por Modelo

Agrupamento sem o uso de `VIEW`:

In [56]:
SELECT Modelo, COUNT(*) Numero_Taxis
       FROM taxi
       GROUP BY Modelo;

org.h2.jdbc.JdbcSQLException:  Table "TAXI" not found; SQL statement

### Tabela de Total de Táxis por Modelo

Transformando o agrupamento na tabela `Contagem_Modelo` com o `VIEW`:

In [57]:
CREATE VIEW Contagem_Modelo AS
SELECT Modelo, COUNT(*) Numero_Taxis
       FROM taxi
       GROUP BY Modelo;

SELECT * FROM Contagem_Modelo;

org.h2.jdbc.JdbcSQLException:  Table "TAXI" not found; SQL statement

Usando a tabela criada com a `VIEW`:

In [58]:
SELECT MAX(Numero_Taxis) FROM Contagem_Modelo;

org.h2.jdbc.JdbcSQLException:  Table "CONTAGEM_MODELO" not found; SQL statement

## VIEW se auto-atualiza

A view é um recorte dinâmico, ela se auto-atualiza quando a tabela original é modificada.

In [59]:
INSERT INTO Taxi VALUES ('KMN3412', 'Chevrolet', 'Corsa', 2001, 'QJ572345');
SELECT * FROM Contagem_Modelo;

org.h2.jdbc.JdbcSQLException:  Table "TAXI" not found; SQL statement